In [9]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio
from electoralyze import region
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import RegionMocked, read_true_geometry, read_true_metadata
from electoralyze.region.redistribute.mapping import _create_intersection_area_mapping, create_region_mapping_base

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# read_true_geometry("region_a").pipe(to_geopandas).explore()

In [22]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("region_a"),
    geometry_to=read_true_geometry("region_b"),
)
intersection_area

region_a,region_b,intersection_area
str,str,f64
"""M""","""A""",0.25
"""M""","""B""",0.75
"""N""","""A""",0.25
"""N""","""C""",0.75
"""O""","""A""",0.75
"""O""","""B""",0.25
"""P""","""A""",0.75
"""P""","""C""",0.25


region_b,intersection_area
str,f64
"""B""",1.0
"""A""",2.0
"""C""",1.0


In [34]:
(
    read_true_geometry("region_b")
    .with_columns(
        st.geom("geometry").st.area(),
        pl.lit(None).cast(pl.String).alias("region_a"),
    )
    .join(
        intersection_area.group_by("region_b").agg(pl.col("intersection_area").sum().alias("intersected_area")),
        on="region_b",
    )
    .select(pl.col("region_a"), pl.col("region_b"), pl.col("geometry").sub(pl.col("intersected_area")))
)

region_a,region_b,geometry
str,str,f64
null,"""C""",1.0
null,"""A""",0.0
null,"""B""",1.0
